In [77]:
# import necessary packages to search for targets with continuous observation
import numpy as np
import pandas as pd
import lightkurve as lk  # main package used here
from astropy.coordinates import Angle

In [22]:
# this function is designed to search targets from the data set
# with given target WISE ID and returns available sectors of data
def search_target(target_name):
    # subtract the substring with char index
    RA_Hour = target_name[1:3]
    RA_Min = target_name[3:5]
    RA_Sec = target_name[5:10]
    RA_String = RA_Hour+'h'+RA_Min+'m'+RA_Sec+'s'
    # obtain the hour angle with astroPy function Angle()
    RA = str(np.around(Angle(RA_String).degree, 7))

    # similar to the procedures in RA
    DEC_Degree = target_name[10:13]
    DEC_Min = target_name[13:15]
    DEC_Sec = target_name[15:19]
    DEC_String = DEC_Degree+'d'+DEC_Min+'m'+DEC_Sec+'s'
    DEC = np.around(Angle(DEC_String).degree, 7)
    if DEC > 0:
        DEC = '+' + str(DEC)
    else:
        DEC = str(DEC)
    targetString = RA + DEC
    pfs = lk.search_tesscut(targetString)
    return pfs

In [88]:
# this function takes target pixel files and check whether the 
# corresponding object has continuous observation across sectors
def check_continuity(sec_num, pixel_file):
    # initialize the continuity flag as false
    continuous = False
    # traverse all missions
    for i in range(len(pixel_file.mission)-sec_num):
        # extract the sector number from the mission string
        curr_sec = int(pixel_file.mission[i].split()[-1])
        next_sec = int(pixel_file.mission[i+sec_num].split()[-1])
        # if the next sector is numerically adjacent to the current
        # sector, break the for loop and return true
        if next_sec-curr_sec == sec_num:
            continuous = True
            break
        else:
            continue
    return continuous

In [107]:
# this function traverses all targets in a given object type
# and return a list of targets with continuous observation
def search_continuity(ID, type):
    list = []

    for i in range(len(ID)):
        target_name = ID[i]
        pfs = search_target(target_name)
        if len(pfs) > 1:
            continuous = check_continuity(sec_num=2, pixel_file=pfs)
            if continuous:
                list.append([target_name, type])

    return list

In [9]:
# this function returns pandas dataframe from the TESS spreadsheet
def read_tess(sheetName):
    # open the Excel sheet that restores the tess data
    TESS = pd.ExcelFile('TESSRechecks_ALLDATA.xlsx')
    # select specified sheets
    sheet = pd.read_excel(TESS, sheetName)
    # extract only the list of WISE ID from the sheet
    return sheet['WISEID']

In [23]:
DD = read_tess('DebrisDisk').dropna()
EDD = read_tess('ExtremeDebrisDisk').dropna()
LateType = read_tess('LateTypeDisk').dropna()
Herbig = read_tess('HerbigAeBe+CBe').dropna()
YSO = read_tess('YSOE+YSOM').dropna()

In [108]:
DD_list = search_continuity(DD, "Debris Disk")
EDD_list = search_continuity(EDD, "Extreme Debris Disk")
LateType_list = search_continuity(LateType, "Late Type Disk")
Herbig_list = search_continuity(Herbig, "HerbigAeBe")
YSO_list = search_continuity(YSO, "YSO")

In [109]:
combined_list = [*DD_list, *EDD_list, *LateType_list, *Herbig_list, *YSO_list]

In [110]:
print(len(combined_list))

49
